I had a hard time with Selenium, it seems a lot has changed and chatGPT was helpful to me during this assignment in figuring out the correct things to download and the new commands in Selenium. 

In [199]:
from selenium import webdriver 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
#The way prof showed us did not work, this is the workaround according to chatgpt
from webdriver_manager.chrome import ChromeDriverManager #Keeps things up to date automatically according to AI
from config1 import UNAME, PASSWORD #Don't need the full path anymore
from time import sleep

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

In [201]:
sleep(15) #Being nice, I am allowing 15 seconds to rest their servers

driver.get("https://mastodon.social/auth/sign_in")


In [203]:
#That worked, Now to input my username and password from the config file
log_form = driver.find_element(By.ID, 'user_email').send_keys(UNAME) #find_element_by_ was discontinued according to chatGPT, this was the solution

In [205]:
#And that worked! Now same thing for the password
pass_form = driver.find_element(By.ID, 'user_password').send_keys(PASSWORD)
#And click the sign in button
pass_button = driver.find_element(By.TAG_NAME, 'button').click()

Turns out, my problem was in the function definition, but also there was a problem because the only the first 2 posts pulled the entire html. All the other posts in my original list were placeholder posts, so chatgpt helped me find the right code for scrolling so that the posts would fully load and the html could be scraped down. 

In [331]:
all_posts = []       # master list
seen_ids = set() 

In [237]:
from selenium.webdriver.common.by import By

feed = driver.find_element(By.CSS_SELECTOR, "div.scrollable.scrollable--flex")

What I did here was run the cell below and then use the for i, p loop to check how many posts there are and if the full HTML was pulled

In [357]:
for i in range(100):
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", feed) #This scrolls 100 times with a 1.5 second sleep for loading the posts
    driver.execute_script("arguments[0].dispatchEvent(new Event('scroll'))", feed)
    sleep(1.5)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    posts = soup.find_all("article")

    for post in posts: #This code block takes all of the unique posts with full HTML and saves them to list all_posts
        post_id = post.get("data-id")
        if post_id not in seen_ids:
            seen_ids.add(post_id)
            if len(str(post)) > 6000: #I learned that posts with over 6000 characters were the full HTML, anything less was a place holder
                all_posts.append(post) 


In [359]:
for i, p in enumerate(all_posts):
    print(i, len(str(p))) #Checking how many posts are in the list and double checking to ensure the full HTML has been pulled

0 6286
1 6300
2 6158
3 6280
4 6194
5 6223
6 7428
7 7412
8 7143
9 7278
10 7209
11 7363
12 7178
13 7275


Yay, all of that worked! This week has been stressful. Now we just need to pull this down

I am going to define a new function that will grab the post and collect the poster name, username, time posted, and body of the post 

In [361]:
def grabpost(post): #figured out what was wrong here
    name = post.find("strong", class_="display-name__html").text
    username = post.find('span', class_='display-name__account').text
    time = post.find('time').text
    body = post.find("div", class_='status__content__text status__content__text--visible translate').text

            
    return {'name': name, 'username': username,
          'time_posted':time, 'post_body': body}

Ok now I have the function, now I need to build the scraper. I need to figure out how to tell it to iterate over the posts

In [363]:
df = pd.DataFrame() #initialize a dataframe

In [365]:
#Ok now we have our posts, lets grab out the relevant information, don't need sleep since the full HTML is already in a list
for post in all_posts:
    postOfInterest = grabpost(post)
    df = pd.concat([df, pd.DataFrame([postOfInterest])], ignore_index=True)

In [367]:
df.to_csv('MastadonFirst14.csv')

I did it!

In [371]:
driver.quit() #Now we can close the driver